In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta  # Technical Analysis library
from sklearn.metrics import mean_squared_error, r2_score
import joblib 


# implement price prediction using lightgbm and LSTM

#  FOR NEW IMPLEMENTING LIGHTGBM


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 2000)  # max characters per column

# Download stock data (example: Apple)
ticker = "AAPL"
df = yf.download(ticker, start="2024-01-01", end="2024-12-10")
df = df.reset_index()

# df.head(5)     # should be (2264,)

new_df = pd.DataFrame()

new_df['Close'] = df['Close']['AAPL']
new_df['High'] = df['High']['AAPL']
new_df['Open'] = df['Open']['AAPL']
new_df['Low'] = df['Low']['AAPL']
new_df['Volume'] = df['Volume']['AAPL']
new_df['Date'] = pd.to_datetime(df['Date'])
df = new_df
# -------------------------
# 2. Basic Price Features
# -------------------------










# -------------------------
# 2. Basic Price Features
# -------------------------
df['return'] = df['Close'].pct_change()
df['daily_range'] = df['High'] - df['Low']
df['close_open_diff'] = df['Close'] - df['Open']


# Lag features (past only)
for lag in range(1, 6):
    df[f'Close_lag_{lag}'] = df['Close'].shift(lag)
    df[f'Volume_lag_{lag}'] = df['Volume'].shift(lag)
    df[f'return_lag_{lag}'] = df['return'].shift(lag)

    
# Rolling features (past only)
for window in [5, 10, 20]:
    df[f'MA_{window}'] = df['Close'].shift(1).rolling(window).mean()
    df[f'std_{window}'] = df['Close'].shift(1).rolling(window).std()


    
# RSI (past only)
df['RSI_14'] = ta.momentum.RSIIndicator(df['Close'].shift(1), window=14).rsi()



# -------------------------
# 6. Date/Time Features
# -------------------------
df['day_of_week'] = df['Date'].dt.dayofweek
df['month'] = df['Date'].dt.month





# Target
df['target_close_next'] = df['Close'].shift(-1)
df.dropna(inplace=True)

df = df.set_index('Date')




# Drop columns that are not features
drop_cols = ['target_close_next']  # remove targets from X
X = df.drop(columns=drop_cols)

# # Load the saved model
lgb_model = joblib.load("lightgbm_stock_model.pkl")
print("✅ Model loaded!")

y_pred = lgb_model.predict(X)
print("Predictions:", len(y_pred))
print(df['target_close_next'].values)
print(y_pred)


rmse = np.sqrt(mean_squared_error(df['target_close_next'].values, y_pred))
r2 = r2_score(df['target_close_next'].values, y_pred)
print(f'RMSE: {rmse:.4f}, R2: {r2:.4f}')




/tmp/ipykernel_16059/2607806152.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2024-01-01", end="2024-12-10")
[*********************100%***********************]  1 of 1 completed

✅ Model loaded!
Predictions: 216
[185.29127502 184.28974915 186.10437012 187.71078491 187.81985474
 186.73902893 187.50352478 185.81561279 183.72067261 182.83700562
 182.54910278 181.01013184 180.26547241 181.02006531 183.05545044
 181.21865845 179.86834717 181.3278656  180.1264801  179.46125793
 178.37902832 173.85154724 168.90704346 167.91416931 167.79504395
 169.51271057 171.51831055 171.99485779 169.90986633 171.76651001
 171.38923645 172.48139954 174.8245697  177.39608765 170.14814758
 171.051651   169.6318512  168.5        172.07429504 170.25735474
 168.81770325 167.63618469 168.44039917 167.61631775 168.37089539
 167.24897766 168.46026611 166.58372498 173.79196167 175.29121399
 171.45872498 168.17233276 166.8021698  165.84901428 163.82356262
 164.65756226 165.71000671 167.81491089 168.67871094 168.09289551
 172.26293945 169.11555481 168.09289551 171.79631042 182.07249451
 180.41442871 181.0994873  181.43707275 183.25402832 181.99137878
 185.20269775 186.34603882 188.62278748 188